<a href="https://colab.research.google.com/github/j0gea/AI-data-competition-Wisdomtooth/blob/main/moldel_list.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 모델

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import timm

ModuleNotFoundError: ignored

#### 파이토치
- torchvision.models.모델명

#### timm
- timm.create_model('모델명')

- 깔린 모델들 /USER/models.py
- 모델들 샘플 폴더 /USER/model_folder

### (0) 기본 샘플 코드

- 모델: densenet121(아래 샘플. 하지만 혼용됩니다(아마?))


In [ ]:
class BaselineModel(nn.Module):
    def __init__(self):
        super(BaselineModel, self).__init__()

        self.model = torchvision.models.densenet121(pretrained=True)
        n_features = self.model.classifier.in_features
        self.fc = nn.Sequential(
            nn.Linear(n_features, 512),
            nn.LeakyReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 1)
        )
        self.model.classifier = self.fc

    def forward(self, x):
        x = self.model(x)
        return x

In [ ]:
# Fully Connected Layer 구조
densenet121 = torchvision.models.densenet121(pretrained=True)
densenet121.classifier

### (1) convnextv2 (224)

- convnextv2_large
- convnextv2_base
- convnextv2_tiny (아래 코드)

가중치 적용 (fcmae, ft, in22k, in1k)
- convnextv2_large.fcmae_ft_in22k_in1k
- convnextv2_base.fcmae_ft_in22k_in1k
- convnextv2_tiny.fcmae_ft_in22k_in1k

In [ ]:
class BaselineModel(nn.Module):
    def __init__(self):
        super(BaselineModel, self).__init__()

        self.model = timm.create_model('convnextv2_tiny', pretrained=True)
        n_features = self.model.head.fc.in_features
        self.fc = nn.Sequential(
            nn.Linear(n_features, 512),
            nn.LeakyReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 1)
        )
        self.model.head.fc = self.fc

    def forward(self, x):
        x = self.model(x)
        return x

In [ ]:
# Fully Connected Layer 구조
convnextv2 = timm.create_model('convnextv2_tiny', pretrained=True)
convnextv2.head.fc

### (2) SE-ResNeXt

- seresnext50_32x4d (224 * 224)
- seresnextaa101d_32x8d

- seresnext50_32x4d.racm_in1k
- seresnextaa101d_32x8d.sw_in12k_ft_in1k_288 (320 or 228)

In [ ]:
class BaselineModel(nn.Module):
    def __init__(self):
        super(BaselineModel, self).__init__()

        self.model = timm.create_model('seresnext50_32x4d', pretrained=True)
        n_features = self.model.fc.in_features
        self.fc = nn.Sequential(
            nn.Linear(n_features, 512),
            nn.LeakyReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 1)
        )
        self.model.fc = self.fc

    def forward(self, x):
        x = self.model(x)
        return x

In [ ]:
model = timm.create_model('seresnext50_32x4d', pretrained=True)
model.fc

### (3) EfficientNet

- B0 224
- B1 240
- B2 260
- B3 300
- B4 380
- B5 456
- B6 528
- B7 600

In [ ]:
from efficientnet_pytorch import EfficientNet

In [ ]:
class BaselineModel(nn.Module):
    def __init__(self):
        super(BaselineModel, self).__init__()

        self.model = EfficientNet.from_pretrained('efficientnet-b4')
        n_features = self.model._fc.in_features
        self.fc = nn.Sequential(
            nn.Linear(n_features, 512),
            nn.LeakyReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, 1)
        )
        self.model._fc = self.fc

    def forward(self, x):
        x = self.model(x)
        return x

In [ ]:
EfficientNet = EfficientNet.from_pretrained('efficientnet-b4')
EfficientNet._fc

### (4)ViT

In [ ]:
# 이진 분류여서 num_classes=1로 설정(이것도 추가한 부분)

In [ ]:
import timm
from torch.optim import lr_scheduler

In [ ]:
class BaselineModel(nn.Module):
    def __init__(self,num_classes=1):
        super(BaselineModel, self).__init__()
        self.model = timm.create_model('vit_base_patch16_224',pretrained=True, num_classes=1)
        # n_features = self.model.classifier.in_features
        # self.fc = nn.Sequential(
        #     nn.Linear(n_features, 512),
        #     nn.LeakyReLU(),
        #    nn.Dropout(0.5),
        #    nn.Linear(512, 1)
        #)
        #self.model.classifier = self.fc

    def forward(self, x):
        x = self.model(x)
        return x

In [ ]:
# 모델 초기화 및 criterion, optimizer 설정 부분에 exp_lr_schedular 추
# Initialize the model, loss function, and optimizer
#model = BaselineModel().to(device)
#criterion = nn.BCEWithLogitsLoss()
#optimizer = optim.Adam(model.parameters(), lr=0.00001)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)


---

In [ ]:
class BaselineModel(nn.Module):
    def __init__(self,num_classes=1):
        super(BaselineModel, self).__init__()
        self.model = timm.create_model('vit_base_patch16_224',pretrained=True, num_classes=1)
        n_features = self.model.head.in_features
        self.model.fc_norm = nn.Sequential(nn.Linear(n_features, 512), nn.LeakyReLU())
        self.model.head_drop = nn.Dropout(0.5)
        self.model.head = nn.Linear(512, 1)

    def forward(self, x):
        x = self.model(x)
        return x

# 기타 코드

### early stpping code

In [ ]:
# Train the model

# early stopping code
early_stopping_epochs = 5
best_loss = float('inf')
early_stop_counter = 0

train_loss_list, val_loss_list, val_metric_list = [], [], []
for epoch in range(num_epochs):
    train_losses = train(model, train_loader, criterion, optimizer, device)
    val_losses, val_metrics = valid(model, val_loader, criterion, device)
    print('Epoch {}, Train Loss: {:.4f}'.format(epoch+1, np.mean(train_losses)))
    print('Epoch {}, Train Loss: {:.4f}, Valid Loss: {:.4f}, Valid Metric: {:.4f}'.format(epoch+1, np.mean(train_losses), np.mean(val_losses), np.mean(val_metrics)))

    # Check for early stopping start
    mean_val_loss = np.mean(val_losses)
    val_metric = np.mean(val_metrics)

    if mean_val_loss < best_loss:
        best_loss = mean_val_loss
        early_stop_counter = 0
        # Save model
        torch.save(model.state_dict(), './model.pth')
    else:
        early_stop_counter += 1

    if early_stop_counter >= early_stopping_epochs:
        print(f'Early stopping at epoch {epoch+1}')
        break
    # Check for early stopping end


### optuna

코드 위치: /USER/YJtest/sample.py

코드 위치 :/USER/YJtest/Convnext_baseline/train.py

In [ ]:
import optuna

In [ ]:
def objective(trial):
    num_epochs = trial.suggest_int('num_epochs', 5, 50)
    num_batches = trial.suggest_int('num_batches', 16, 128)

    train_transform = transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomRotation(30),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    val_transform = transforms.Compose([
        transforms.Resize(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])


    for train_idx, val_idx in skf.split(df, df['risk']):
        train_df = df.iloc[train_idx]
        val_df = df.iloc[val_idx]
        break

    train_dataset = BaselineDataset(train_df, transform=train_transform)
    val_dataset = BaselineDataset(val_df, transform=val_transform)

    train_loader = DataLoader(train_dataset, batch_size=num_batches, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=num_batches, shuffle=False)
    model = BaselineModel().to(device)
    criterion = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.00001)

    for epoch in range(num_epochs):
        model.train()
        losses = []
        for inputs, labels in tqdm(train_loader):
            inputs, labels = inputs.to(device), labels.float().to(device)
            #inputs, labels = inputs.to(device), labels.float().to(device)
            optimizer.zero_grad()
            outputs = model(inputs).view(-1)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            losses.append(loss.item())
        print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {np.mean(losses)}')

    return np.mean(losses)


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=2)

best_trial = study.best_trial
print(f"Best trial - Loss: {best_trial.value}, Params: {best_trial.params}")

### F1 예측 코드
- 조금 더 표본이 필요함
- 당연하지만 정확하지 않음
- 사용할때 예측코드 복사해서 사용하는걸 추천

- output: **F1test**_submission.csv 인데 **절대!! 실수로 제출하지 않도록!!**

- True Positive (TP) : “high”으로 정확하게 예측된 샘플의 수 (예측 h 실제 h)
- False Positive (FP) : “high”으로 잘못 예측된 샘플의 수 (예측 h 실제 l
- True Negatives (TN) : “low”으로 올바르게 예측된 샘플의 수 (예측 l 실제 l)
- False Negatives (FN) : “low”으로 잘못 예측된 샘플의 수 (예측l 실제h)

precision = high로 정확하게 예측된 샘플의 수 / 전체 high로 예측 한 수
TP / TP+FP

recall = high로 정확하게 예측된 샘플의 수 / 실제 high의 수
TP / TP+FN

 2 * ((precision * recall) / (precision + recall))

 각 레이블 별로 계산, 평균내기(low high 계산)

In [ ]:
model.eval()
preds_list = []
with torch.no_grad():
    for image in tqdm(test_loader):
        image = image.to(device)
        outputs = model(image).view(-1)

        preds = torch.sigmoid(outputs).round()
        preds_list += preds.cpu().numpy().tolist()

# start here
test_df['pre_risk'] = preds_list
test_df['pre_risk'] = test_df['pre_risk'].apply(lambda x: 'high' if x == 1 else 'low')
test_df['result'] = test_df['risk'] == test_df['pre_risk']

#high f1
high = test_df.loc[test_df.pre_risk == 'high'] # TP+FP
Rhigh = test_df.loc[test_df.risk == 'high'] # TP+FN

TP = high.loc[high.risk == 'high'] # TP

precision = len(TP) / len(high)
recall = len(TP) / len(Rhigh)
F1_high = 2 * ((precision * recall) / (precision + recall))

#print(F1_high)

#low f1
low = test_df.loc[test_df.pre_risk == 'low'] # TP+FP
Rlow = test_df.loc[test_df.risk == 'low'] # TP+FN

TP = low.loc[low.risk == 'low'] # TP

precision = len(TP) / len(low)
recall = len(TP) / len(Rlow)
F1_low = 2 * ((precision * recall) / (precision + recall))

#print(F1_low)

F1 = (F1_high + F1_low) / 2

print("F1 test score : ", F1)

test_df.to_csv('./F1test_submission.csv', index=False)
# End here

아래 두 위치를 train으로 바꾸어주어야 합니다
그거 기반으로 하는거라

In [ ]:
img_fname = f'/DATA/train/images/{img_name}'
test_df = pd.read_csv(f'/DATA/train/train.csv')

### Gradient Accumulation

참고 사이트

- https://medium.com/huggingface/training-larger-batches-practical-tips-on-1-gpu-multi-gpu-distributed-setups-ec88c3e51255 (code)

- https://gist.github.com/thomwolf/ac7a7da6b1888c2eeac8ac8b9b05d3d3#file-gradient_accumulation-py (위 코드 깃허브)

- https://velog.io/@nawnoes/Pytorch%EB%A1%9C-%ED%81%B0-%EB%AA%A8%EB%8D%B8-%ED%95%99%EC%8A%B5%EC%8B%9C-%EC%96%B4%EB%96%BB%EA%B2%8C-%EB%B0%B0%EC%B9%98-%EC%82%AC%EC%9D%B4%EC%A6%88%EB%A5%BC-%EB%8A%98%EB%A6%B4%EC%88%98-%EC%9E%88%EC%9D%84%EA%B9%8C (한국어 이론)

In [ ]:
# Define the training function
# accumulation_steps (How much to slice)
def train(model, train_loader, criterion, optimizer, device, accumulation_steps):
    model.train()
    losses = []
    for i, (inputs, labels) in enumerate(tqdm(train_loader)):
        inputs, labels = inputs.to(device), labels.float().to(device)

        outputs = model(inputs).view(-1)
        # 1 d vecter (view(-1))

        loss = criterion(outputs, labels)
        loss = loss / accumulation_steps
        loss.backward()

        if (i + 1) % accumulation_steps == 0:
            optimizer.step()
            model.zero_grad()

        losses.append(loss.item())
    return losses

In [ ]:
# hyperparameters
# num_batches  = want_num_batches / accumulation_steps
num_epochs = 40
num_batches = 16 # same as 32

# How much to slice
accumulation_steps = 2

In [ ]:
train_losses = train(model, train_loader, criterion, optimizer, device, accumulation_steps)
# added accumulation_steps

### 결과 txt파일 내놓기

In [ ]:
f = open("result.txt",'w')

data = 'Epoch {}, Train Loss: {:.4f}, Valid Loss: {:.4f}, Valid Metric: {:.4f}\n'.format(epoch+1, np.mean(train_losses), np.mean(val_losses), np.mean(val_metrics))
f.write(data)

f.close()

---